In [105]:
import numpy as np
import pandas as pd
import os.path
import random

from cvxopt import matrix, solvers
from scipy.special import expit as sigmoid
from sklearn.model_selection import train_test_split

N_TRIALS=10
MIN_MULTIPLIER_THRESHOLD=1e-8

In [107]:
def n_cross_val(A, folds):
    n=len(A)//folds
    for i in range(0, len(A), n):
        yield (A[:i]+A[i+n:], A[i:i + n])

In [108]:
class SoftMarginSvm:
    def __init__(self, X, y, C):
        self.C = C
        self.datapoints, self.dimensions = X.shape
        self.alphas=None
        self.support_vectors=None
        self.support_vector_labels=None
        self.bias = 0
        self.build(X,y)
    
    def _lagrangian(self, X, y):
        # find the gram matrix
        K=y.reshape(-1,1)*X
        K = np.dot(K, K.T)
        
        P=matrix(K)
        q=matrix( np.ones((self.datapoints,1)) * -1 )
        G1=matrix( np.eye(self.datapoints) * -1 )
        H1=matrix( np.zeros(self.datapoints) )
        G2=matrix( np.eye(self.datapoints) )
        H2=matrix( np.ones((self.datapoints,1)) * self.C )
        G=matrix( np.vstack((G1, G2)) )
        h=matrix( np.vstack((H1, H2)) )
        A=matrix( y.reshape(1,-1) )
        b=matrix(np.zeros(1))
        
        solution = solvers.qp(P, q, G, h, A, b)
        #solution = solvers.qp(P, q, G1, H1, A, b)
        alphas = np.array(solution['x'])
        #print("alphasize=", alphas.shape)
        #print("alphas=",alphas)
        return alphas
    
    def _find_support_vectors(self, X, y, multipliers):
        sv_indices = multipliers > MIN_MULTIPLIER_THRESHOLD
        #sv_indices = multipliers > 0 
        # print("nonzero multipliers::", (multipliers > MIN_MULTIPLIER_THRESHOLD).sum())
        
        self.alphas = multipliers[sv_indices]
        self.support_vector_labels = y[sv_indices]
        self.support_vectors = X[np.where(sv_indices), :]
        #self.support_vector_labels = self.support_vector_labels.reshape(-1,1)
        #print("support_vector_labels.shape=",self.support_vector_labels.shape)
        #print("support_vectors.shape=",self.support_vectors.shape)
        # w = sum (alpha * y * X)
        self.w=np.sum(multipliers * y * X, axis = 0)
        self.w = self.w.reshape(-1,1)
        # setting bias as the (yi-wT.xi) for i where alpha>0
        #print("self.w.shape",self.w.shape)
        #b = []
        #for e in self.support_vectors:
        #    b.append(self.support_vector_labels - np.dot(self.w.T,e))
        #print("b.shape=",b.shape)
        #self.bias = np.mean(b)
        #print("bias=",self.bias)
        
    def predict(self,X):
        result =[]
        for x in X:
            res = np.sign(np.dot(self.w.T,x))
            result.append(res)
        return result
    
    def calc_error(self, pred, target):
        correctness = np.array([yn == y for (yn,y) in zip(pred, target)])
        return 1 - (np.sum(correctness) / target.size)
    
    def validate(self, Xnew, Ynew):
        mypredictions=self.predict(Xnew)
        return self.calc_error(mypredictions, Ynew)
    
    def _train(self, X, y):
        lagr_multipliers = self._lagrangian(X, y)
        self._find_support_vectors(X, y, lagr_multipliers)
    
    def build(self, X, y):
        print()
        print("Training the SVM..")
        self._train(X, y)

In [109]:
def myDualSVM(filename, Clist):
    assert os.path.isfile(filename) and os.access(filename, os.R_OK)
    df=pd.read_csv(filename, sep=',', header = None)
    #print(df.shape)          #(2000, 785)
    
    data = df.as_matrix()
    y=data[:, 0]
    X=data[:, 1:]
    
    del df,data
    
    def reclassify(labels, a, b):
        f=np.vectorize(lambda x: -1.0 if x==a else 1.0)
        return f(labels)
    
    original_classes = np.unique(y)
    assert len(original_classes)==2
    y=reclassify(y, original_classes[0], original_classes[1])
    y=np.reshape(y, [X.shape[0], 1])
    
    #print(np.unique(y))      #[-1, 1]
    #print(y.shape)           #(2000, 1)
    #print(X.shape)           #(2000, 784)
    
    errormatrix = np.zeros(N_TRIALS)
    
    for t in range(N_TRIALS):
        X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2)
        Y_train = np.reshape(Y_train, [X_train.shape[0], 1])
        Y_test = np.reshape(Y_test, [X_test.shape[0], 1])
    
        for C in Clist:
            svm=SoftMarginSvm(X_train, Y_train, C)
            errormatrix[t] = svm.validate(X_test, Y_test)
            print("Trial %d : Classification Error for C = %s : %0.4f\n"%(t+1, C, errormatrix[t]))
        else:
            #pretty plot pickle
            # TODO
            pass
    errmu = np.mean(errormatrix)
    errsigma = np.std(errormatrix, ddof=1)
    print("Mean test-error"+str(errmu))
    print("Std test-error"+str(errsigma))

In [110]:
myDualSVM('./MNIST-13.csv', [10e-8, 10e-7, 10e-6, 10e-5])


Training the SVM..
     pcost       dcost       gap    pres   dres
 0: -2.9257e+01 -1.4413e-02  9e+03  9e+01  9e-09
 1: -4.5404e-01 -1.1047e-02  1e+02  1e+00  9e-09
 2: -1.7313e-02 -2.0088e-03  4e+00  4e-02  3e-10
 3: -1.5041e-03 -5.0782e-04  2e-01  2e-03  2e-11
 4: -1.6359e-04 -2.9465e-04  1e-02  1e-04  1e-12
 5: -2.4716e-05 -2.6024e-04  9e-04  7e-06  7e-14
 6: -1.3799e-05 -1.4471e-04  2e-04  2e-06  2e-14
 7: -8.9449e-06 -5.4383e-05  7e-05  4e-07  5e-15
 8: -8.6717e-06 -2.1555e-05  2e-05  7e-08  3e-15
 9: -9.1818e-06 -1.7896e-05  1e-05  4e-08  2e-15
10: -9.7775e-06 -1.5314e-05  6e-06  2e-08  2e-15
11: -1.0170e-05 -1.3919e-05  4e-06  1e-08  2e-15
12: -1.0402e-05 -1.3199e-05  3e-06  8e-09  2e-15
13: -1.0678e-05 -1.2409e-05  2e-06  4e-09  2e-15
14: -1.0908e-05 -1.1818e-05  9e-07  2e-09  2e-15
15: -1.1080e-05 -1.1467e-05  4e-07  5e-10  2e-15
16: -1.1196e-05 -1.1258e-05  6e-08  5e-11  2e-15
Optimal solution found.
alphasize= (1600, 1)
alphas= [[  1.36733914e-11]
 [  2.07010048e-10]
 [  9.

 7: -1.7444e-04 -4.4291e-03  6e-03  2e-05  2e-13
 8: -8.0813e-05 -1.3837e-03  2e-03  6e-06  8e-14
 9: -5.3750e-05 -4.3728e-04  5e-04  2e-06  4e-14
10: -5.3752e-05 -1.6688e-04  1e-04  4e-07  3e-14
11: -4.7478e-05 -1.3889e-04  1e-04  9e-08  3e-14
12: -5.8662e-05 -9.6412e-05  4e-05  3e-08  3e-14
13: -6.1938e-05 -8.4458e-05  2e-05  7e-10  3e-14
14: -6.7227e-05 -7.4634e-05  7e-06  2e-10  3e-14
15: -6.9449e-05 -7.0760e-05  1e-06  2e-11  3e-14
16: -6.9945e-05 -7.0043e-05  1e-07  2e-14  3e-14
Optimal solution found.
alphasize= (1600, 1)
alphas= [[  8.01639337e-12]
 [  2.76882089e-11]
 [  7.68625215e-12]
 ..., 
 [  5.97226332e-11]
 [  4.16431577e-11]
 [  1.16998490e-06]]
self.w.shape (784, 1)
Trial 2 : Classification Error for C = 0 : 0.0150


Training the SVM..
     pcost       dcost       gap    pres   dres
 0: -3.4160e+01 -1.7273e-02  1e+04  1e+02  8e-09
 1: -5.4538e-01 -1.2463e-02  1e+02  2e+00  8e-09
 2: -1.6517e-02 -1.4567e-03  4e+00  4e-02  2e-10
 3: -1.0896e-03 -4.4990e-04  2e-01  2e-03

15: -6.4107e-05 -6.4121e-05  1e-08  7e-20  3e-14
Optimal solution found.
alphasize= (1600, 1)
alphas= [[  8.05531547e-12]
 [  3.21205511e-12]
 [  5.17762401e-12]
 ..., 
 [  3.27370538e-12]
 [  2.62060653e-11]
 [  1.18438825e-11]]
self.w.shape (784, 1)
Trial 4 : Classification Error for C = 0 : 0.0150


Training the SVM..
     pcost       dcost       gap    pres   dres
 0: -3.3587e+01 -2.9408e-01  9e+03  1e+02  8e-09
 1: -6.6169e-01 -2.8509e-01  2e+02  2e+00  8e-09
 2: -1.7900e-02 -2.7593e-01  3e+00  3e-02  2e-10
 3: -6.9898e-03 -2.0887e-01  5e-01  3e-03  2e-11
 4: -3.3931e-03 -8.8152e-02  1e-01  8e-04  6e-12
 5: -1.8904e-03 -4.7217e-02  7e-02  4e-04  3e-12
 6: -4.1880e-04 -1.7933e-02  3e-02  1e-04  1e-12
 7: -1.4471e-04 -4.1088e-03  6e-03  2e-05  2e-13
 8: -6.4524e-05 -1.2059e-03  2e-03  5e-06  7e-14
 9: -4.3532e-05 -3.5903e-04  4e-04  1e-06  4e-14
10: -4.7241e-05 -1.5209e-04  1e-04  3e-07  3e-14
11: -4.8989e-05 -9.7688e-05  5e-05  6e-19  3e-14
12: -5.6613e-05 -7.8730e-05  2e-05  7e-19

 1: -4.5354e-01 -3.2120e-02  1e+02  1e+00  9e-09
 2: -1.8499e-02 -2.8596e-02  4e+00  4e-02  3e-10
 3: -1.7005e-03 -2.6988e-02  2e-01  2e-03  1e-11
 4: -8.3837e-04 -1.8251e-02  4e-02  3e-04  3e-12
 5: -4.4148e-04 -7.5482e-03  1e-02  7e-05  7e-13
 6: -2.4619e-04 -3.3332e-03  5e-03  3e-05  3e-13
 7: -1.4171e-04 -1.7721e-03  3e-03  1e-05  1e-13
 8: -6.6046e-05 -8.7091e-04  1e-03  4e-06  7e-14
 9: -4.8834e-05 -4.9397e-04  6e-04  2e-06  4e-14
10: -4.2660e-05 -2.0659e-04  2e-04  5e-07  3e-14
11: -4.7168e-05 -1.0814e-04  6e-05  8e-20  3e-14
12: -5.5108e-05 -9.0256e-05  4e-05  7e-20  3e-14
13: -6.3253e-05 -7.1232e-05  8e-06  7e-20  3e-14
14: -6.5343e-05 -6.7587e-05  2e-06  7e-20  3e-14
15: -6.6114e-05 -6.6407e-05  3e-07  8e-20  4e-14
16: -6.6247e-05 -6.6251e-05  4e-09  7e-20  3e-14
Optimal solution found.
alphasize= (1600, 1)
alphas= [[  1.00953444e-12]
 [  1.61465209e-12]
 [  5.64089747e-13]
 ..., 
 [  2.70997103e-12]
 [  1.99723894e-12]
 [  6.43955409e-12]]
self.w.shape (784, 1)
Trial 6 : Cla

 8: -3.1183e-05 -1.2134e-04  1e-04  5e-07  1e-14
 9: -3.2065e-05 -7.0491e-05  5e-05  1e-07  9e-15
10: -3.1572e-05 -5.7668e-05  3e-05  7e-21  1e-14
11: -3.4730e-05 -4.9834e-05  2e-05  7e-21  1e-14
12: -3.4898e-05 -4.9394e-05  1e-05  8e-21  1e-14
13: -3.6354e-05 -4.5631e-05  9e-06  8e-21  1e-14
14: -3.8362e-05 -4.1844e-05  3e-06  7e-21  1e-14
15: -3.9280e-05 -4.0403e-05  1e-06  7e-21  1e-14
16: -3.9717e-05 -3.9846e-05  1e-07  8e-21  1e-14
17: -3.9774e-05 -3.9776e-05  2e-09  8e-21  1e-14
Optimal solution found.
alphasize= (1600, 1)
alphas= [[  2.81137915e-13]
 [  5.82001813e-13]
 [  7.38636884e-13]
 ..., 
 [  3.30904567e-13]
 [  5.17983529e-13]
 [  1.91534440e-12]]
self.w.shape (784, 1)
Trial 8 : Classification Error for C = 0 : 0.0050


Training the SVM..
     pcost       dcost       gap    pres   dres
 0: -3.4429e+01 -7.4265e-02  9e+03  1e+02  8e-09
 1: -6.6373e-01 -4.2828e-02  2e+02  2e+00  8e-09
 2: -1.8226e-02 -2.9218e-02  4e+00  4e-02  3e-10
 3: -1.7153e-03 -2.7366e-02  2e-01  2e-03

12: -1.1086e-05 -1.3635e-05  3e-06  6e-09  2e-15
13: -1.1388e-05 -1.2783e-05  1e-06  2e-09  2e-15
14: -1.1655e-05 -1.2161e-05  5e-07  4e-10  2e-15
15: -1.1762e-05 -1.1986e-05  2e-07  2e-10  2e-15
16: -1.1832e-05 -1.1871e-05  4e-08  8e-22  2e-15
Optimal solution found.
alphasize= (1600, 1)
alphas= [[  4.31050652e-11]
 [  4.61419001e-12]
 [  3.31566979e-11]
 ..., 
 [  7.52086365e-12]
 [  2.81559970e-08]
 [  3.56441733e-11]]
self.w.shape (784, 1)
Trial 10 : Classification Error for C = 0 : 0.0125


Training the SVM..
     pcost       dcost       gap    pres   dres
 0: -3.4131e+01 -6.3763e-02  1e+04  1e+02  9e-09
 1: -6.4669e-01 -1.6147e-02  2e+02  2e+00  9e-09
 2: -1.6440e-02 -4.0744e-03  4e+00  4e-02  3e-10
 3: -1.2723e-03 -3.2137e-03  2e-01  2e-03  1e-11
 4: -2.0184e-04 -2.9074e-03  1e-02  1e-04  9e-13
 5: -9.5538e-05 -1.8123e-03  3e-03  2e-05  1e-13
 6: -5.5548e-05 -7.2333e-04  1e-03  5e-06  4e-14
 7: -3.7044e-05 -2.9713e-04  4e-04  2e-06  2e-14
 8: -3.0396e-05 -1.2268e-04  1e-04  4e-0